In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [3]:
#imports data on what news is fake and what news is real
data_fake = pd.read_csv('/Users/jonathancamacho/Desktop/CSprojects/hackathon/src/data/Fake.csv')
data_true = pd.read_csv('/Users/jonathancamacho/Desktop/CSprojects/hackathon/src/data/True.csv')

In [4]:
#classifying fake and real news
data_fake["class"] = 0
data_true['class'] = 1

In [5]:
#checks number of rows and columns in the data set
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [6]:
#manually tests data (?)
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480, 23470, -1):
    data_fake.drop([i], axis = 0, inplace = True)
data_true_manual_testing = data_true.tail(10)
for i in range(21416, 21406, -1):
    data_true.drop([i], axis = 0, inplace = True)


In [7]:
#classifies data set
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

/var/folders/9v/jwwq27814xn00fm688mytq480000gn/T/ipykernel_98480/4059887460.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
/var/folders/9v/jwwq27814xn00fm688mytq480000gn/T/ipykernel_98480/4059887460.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [9]:
#merges the fake and true news data sets into one
data_merge = pd.concat([data_fake, data_true], axis = 0)

In [10]:
#gets rid of irrelevant columns
data = data_merge.drop(['title', 'subject', 'date'], axis = 1)

In [11]:
#cleaning text
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [12]:
#applying function to text column and assigning x/y (?)
data['text'] = data['text'].apply(wordopt)
x = data['text']
y = data['class']

In [14]:
#Defining Training and Testing Data and Splitting Them Into &5 -25 Percent Ratio (?)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

In [15]:
#putting raw data into a matrix
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [17]:
#creaing regression model
from sklearn.linear_model import LogisticRegression

In [18]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [19]:
#checking model accuracy and classification report
pred_lr = LR.predict(xv_test)
LR.score(xv_test, y_test)

0.9876114081996435

In [20]:
#creating decision tree classified model
from sklearn.tree import DecisionTreeClassifier

In [21]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [22]:
#checking model accuracy and classification report
pred_dt = DT.predict(xv_test)
DT.score(xv_test, y_test)

0.9959893048128342

In [23]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [25]:
pred_rf = RF.predict(xv_test)
RF.score(xv_test, y_test)

0.9900178253119429

In [26]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not Fake News"

In [27]:
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    
    return print("\n\nLR Prediction: {} \nDT Prediction:".format(output_label(pred_LR[0]), output_label(pred_DT[0])))


In [28]:
news = str(input())
manual_testing(news)